# Neural Machine Translation (German to English)

## Content <a name='content'></a>

[0. Default Solution](#sec_0)
    
- [0.1 Qualitative Results on Dev Dataset](#sec_0_1)
    
- [0.2 Quantitative Results on Dev Dataset](#sec_0_2)
    
[1. Our Method](#sec_1)

- [1.1 Baseline Method](#sec_1_1)
    
- [1.2 Beam Search](#sec_1_2)
    
- [1.3 Post Processing: Replication Removal](#sec_1_3)
    
- [1.4 Post Processing: \<UNK\> Replacement](#sec_1_4)
    
- [1.5 "Ensemble" of Models](#sec_1_5)
    
[2. Experiments](#sec_2)

- [2.1 Baseline Method](#sec_2_1)
    
- [2.2 Beam Search](#sec_2_2)
    
- [2.3 Replication Removal](#sec_2_3)
    
- [2.4 \<UNK\> Replacement](#sec_2_4)
    
- [2.5 "Ensemble" of Models](#sec_2_5)

- [2.6 Final Results](#sec_2_6)
    
[3.Analysis](#sec_3)


## 0. Default Solution <a name='sec_0'></a> 
[back to content](#content)

The default solution is a seq2seq neural machine translation (NMT) network. The encoder is bi-GRU, and the decoder is GRU. The default solution's problem is that its attention mechanism between the encoder and the decoder is not correctly implemented.

In [2]:
import os, sys
from default import *
directory=os.getcwd()

## 0.1 Qualitative Results on Dev Dataset <a name='sec_0_1'></a>
[back to content](#content)

First, we test the default method. The pre-trained model `seq2seq_E049.pt` is used, and the method is tested on `Dev` dataset. We printed out some translation results as the qualitative results.

In [2]:
os.chdir('../')
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results)[:4999])

1305it [03:35,  6.05it/s]

i was , i had my my , i had my . 
i was in , and i was in . 
she was a woman . she was a . 
and she was , and she was , and she was , and she was , and she was , and she was , and she was , and she was , and she was , and she was , and she was . 
i was , i was , i was . 
and when i had a first patient , , , a patient . 
and i was talking about , and i wanted to talk about , and i wanted to talk about . 
i i i , i i . 
i did n't have it . 
so , to the , , , to the , , , to the , , , and , and , and , and , to the 
i was , , " , " , " , " , " , " 
later , later , later , later , later came later , later came later later later later . 
and i had n't think of the , and i had n't have a 
and i was , to , , , . 
i i was . . 
so , you , you , , you , you , you , you , you , you know , you you , you know , you you . 
and , , " , , , " , " , " 
and the 's the to the , the , the , the . 
that 's what it call . 
and the was the that that was the that i was the . 
so , , as a , , , , , , , , , did

## 0.2 Quantitative Results on Dev Dataset <a name='sec_0_2'></a>
[back to content](#content)

We evaluated the default method with the `BLEU` metric. The default method obtained a poor result (`BLEU = 3.35`): 

In [3]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))
print(bleu(ref_t, results).score)

BLEU = 3.35 34.4/7.6/2.1/0.6 (BP = 0.771 ratio = 0.794 hyp_len = 19766 ref_len = 24902)
3.3529164212375866


## 1. Our Methods <a name='sec_1'></a>
[back to content](#content)

### 1.1 Baseline Method <a name='sec_1_1'></a>
[back to content](#content)

In `baseline method`, we used the same network as the `default method`. The difference is that we fixed the attention mechanism between the encoder and the decoder in `baseline method`.

#### Implementation of Attention Mechanism Between Encoder and Decoder

#### 1.1.1 Computation of Attention Weights

The `AttentionModule` has 3 weight matrices, $W_{enc}$, $W_{dec}$, and $V_{att}$ (which are implemented as the linear layers `nn.Linear`).

For each decoder's hidden state $h^{dec}$, we need to compute its relevance score with regard to encoder's outputs $h_{i}^{enc}$. That is, for the next word to translate, we need to find the most relevant information in the encoded representations $h_{i}^{enc}$:

$score_{i}=W_{enc}(h_{i}^{enc})+W_{dec}(h^{dec})$

Then, we use the `softmax` function to normalized the scores, so that they sum to 1:

$\alpha=softmax(V_{att}tanh(score))$

Here, $\alpha$ is the vector $\alpha=(\alpha_{1},...,\alpha_{n})$ of attention weights. Each $\alpha_{i}$ describes how the original word at position $i$ (encoded as $h_{i}^{enc}$) is relevant to the next word to translate.

The computation of attention weights is implemented in `calcAlpha`.

Initially, the tensor $h^{enc}$ (`encoder_out`) and $h^{dec}$ (`decoder_hidden`) are in the shape of `(seq, batch, dim)`. First, we need to transpose the tensors so that they are in the shape of `(batch, seq, dim)`: 

```
encoder_out=encoder_out.permute(1,0,2)
decoder_hidden=decoder_hidden.permute(1,0,2)
```

Then, we implemented the equation:

$score_{i}=W_{enc}(h_{i}^{enc})+W_{dec}(h^{dec})$

```
scores = self.W_enc(encoder_out)+self.W_dec(decoder_hidden)
```

Then, we implemented the equation:

$\alpha=softmax(V_{att}tanh(score))$

to obtain the attention weights $\alpha$.

```
scores=self.V_att(torch.tanh(scores))
alpha = torch.nn.functional.softmax(scores, dim=1)
```

#### 1.1.2 Computation of Context Vector

After obtaining the attention weights $\alpha$, we calculate the weighted sum of $h_{i}^{enc}$ to get the context vector $c$:

$c=\underset{i}{\Sigma}\alpha_{i}\cdot h_{i}^{enc}$

This is implemented in the `forward` function of `AttentionModule`.

In the `forward` function, we first call the function `calcAlpha` to obtain $\alpha$:

```
alpha = self.calcAlpha(decoder_hidden, encoder_out)
```

Then, we implemented the equation $c=\underset{i}{\Sigma}\alpha_{i}\cdot h_{i}^{enc}$. Note that initially `encoder_out` ($h^{enc}$) is in the shape of `(seq, batch, dim)`, and we need to transpose it so that it is in the shape of `(batch, seq, dim)`.

```
encoder_out=encoder_out.permute(1,0,2)
```

After the transposition, $h_{i}^{enc}$'s index $i$ is in the second dimension (`seq`). To calculate the context vector $c$, we need to take the sum along `dim=1`:

```
context=torch.sum(alpha*encoder_out, dim=1)
```

Now, `context` is in the shape of `(batch, dim)`. In fact, in the implementation the `batch` here is always `1`, so, we can reshape `context` so that it is in the shape of `(1, 1, dim)`:

```
seq, _, dim = encoder_out.shape
context = context.reshape(1, 1, dim)
```

Recall that `alpha` is obtained from `alpha = torch.nn.functional.softmax(scores, dim=1)`, which means that the weights information is in `dim=1`. Now we transpose `alpha` so that the weights information is in `dim=2`:

```
alpha=alpha.permute(2, 0, 1)
```

We can assemble the above implementations in 4 lines:

```
alpha = self.calcAlpha(decoder_hidden, encoder_out)
seq, _, dim = encoder_out.shape
context = (torch.sum(alpha*encoder_out.permute(1,0,2), dim=1)).reshape(1, 1, dim)
return context, alpha.permute(2, 0, 1)
```

See experiment results [here](#sec_2_1).

### 1.2 Beam Search <a name='sec_1_2'></a>
[back to content](#content)

In `Beam Search` method, the network architecture and the attention mechanism are the same as in `baseline` method. Here we implemented beam search to allow better decoding strategies for neural machine translation.

#### 1.2.1 Overview of the Method

Suppose that at time step $t$, given the last translated word $x_{t}$, encoder's output $h^{enc}$, and decoder's hidden state $h_{t}^{dec}$ at time $t$, the decoder returns:

$\tilde{x}_{t+1}, h_{t+1}^{dec}, \alpha_{t} = Decoder(x_{t}, h^{enc}, h_{t}^{dec})$

Here $\tilde{x}_{t+1}$ is a `distribution` of the probability of the next predicted word. For `greedy search` implemented in the `baseline` method, we simply choose the word which maximizes this probability as the next word $x_{t+1}$:

$x_{t+1}=\underset{v\in V}{argmax} \tilde{x}_{t+1}(v)$

So, in greedy decoding, at each time step $t$, we greedily choose the top one candidate word which gets the highest probability for this time step.

In beam search, instead of keeping only the top one candidate word $x_{t}$ at each time step $t$, we keep the top $k$ candidate words, $x_{t,1}, ..., x_{t,k}$, at each time step $t$.

So, `greedy search` is a special case of `beam search` with `k=1`. 

Sometimes, `greedy search` is unable to produce good outcomes, as combining locally optimal choices won't guarantee a globally optimal choice. On the other hand, `exhaustive search` can guarantee the solution's global optimality, however, exhaustively searching over all possibilites is too computationally expensive. So, we want to find a balance between `optimality` and `computational efficiency`, which is why we implemented `beam search`. 

As the algorithm is not as short-sighted as `greedy search`, with `beam search`, we can get better translations. Meanwhile, we can tune the parameter `k` so that the cost of computational resources is not too much.

For each time step $t$, we keep the top $k$ candidate words (the $k$ most probable words), $x_{t,1},...,x_{t,k}$. For each candidate word $x_{t,i}$, we calculate:

$\tilde{x}_{t+1,i}, h_{t+1}^{dec}, \alpha_{t} = Decoder(x_{t,i}, h^{enc}, h_{t}^{dec})$

Here $\tilde{x}_{t+1,i}$ is a `distribution` of the probability of the next predicted word given $x_{t,i}$ is the word at time step $t$.

For each `distribution` $\tilde{x}_{t+1,i}$, we can find the top $k$ candidate words, $x_{t+1,i,1},...,x_{t+1,i,k}$:

$x_{t+1,i,1}=\underset{v\in V}{argmax}\tilde{x}_{t+1,i}(v)$

$x_{t+1,i,2}=\underset{v\in V-\{x_{t+1,i,1}\}}{argmax}\tilde{x}_{t+1,i}(v)$

...

$x_{t+1,i,k}=\underset{v\in V-\{x_{t+1,i,1},...,x_{t+1,i,k-1}\}}{argmax}\tilde{x}_{t+1,i}(v)$

So, for time step $t+1$, we find $k^{2}$ candidate words, $x_{t+1,i,j}(1\leq i\leq k, 1\leq j\leq k)$. From $x_{t+1,i,j}(1\leq i\leq k, 1\leq j\leq k)$ we choose the $k$ most probable words as the $k$ candidate words $x_{t+1,1},...,x_{t+1,k}$ for the time step $k+1$.

Since $x_{t+1,1},...,x_{t+1,k}$ are generated from $x_{t+1,i,j}(1\leq i\leq k, 1\leq j\leq k)$, and $x_{t+1,i,j}$s are generated from $x_{t,i}$, of course we can trace back which $x_{t,i}$ generates the $k$ candidate words at time step $t+1$. Hence, at each time step $t$, we can find the $k$ "so far the most probable" sequences.

We keep generating the $k$ most probable words at each time steps, until we finish the whole sentence. Then, in the $k$ most probable sequences, we pick the most probable one, which is our final translation.

#### 1.2.2 Overview of Implementation

To implement `beam search`, we need to:

1) `Initialization`: For the time step $t=1$, find the $k$ most probable words $x_{1,1},...,x_{1,k}$;

2) `Recursion`: Given that we have the $k$ most probable sequences at time step $t$, generate the $k^{2}$ candidate words at time step $t+1$. Choose the $k$ most probable words from the $k^{2}$ candidate words.

3) `Update`: Given the $k$ most probable words at time step $t+1$, compute the $k$ most probable sequences at time step $t+1$.

Then, we can recursively compute the $k$ most probable sequences at any time step, which completes the task of `beam search`.

#### 1.2.3 Implementation Detail (I): Auxillary Classes

In our implementation, we first define 2 auxillary classes, `Entry` and `Prob_Rank`.

`Entry` stores the $k$ most probable sequences so far at time step $t$:

```
class Entry:
    def __init__(self,idx_seq,hidden_state_seq,alpha_seq,log_prob):
        self.idx_seq=idx_seq # the sequence of words from 1 to t
        self.hidden_state_seq=hidden_state_seq # the sequence of hidden states from 1 to t
        self.alpha_seq=alpha_seq # the sequence of alpha vectors from 1 to t
        self.log_prob=log_prob # logarithm of the probability of the sequence
```

`Prob_Rank` stores the most probable words at time step $t+1$:

```
class Prob_Rank:
    def __init__(self,prev,idx,hidden_state,alpha,log_prob):
        self.prev=prev # this word at time step t+1 is generated from which sequence at time step t
        self.idx=idx # the generated word at time step t+1
        self.hidden_state=hidden_state # hidden state at time step t+1
        self.alpha=alpha # alpha vector at time step t+1
        self.log_prob=log_prob # logarithm of the joint probability of this word and its previous sequence
    def __lt__(self,other):
        # we store candidate words at time step t+1 in a min-heap. We use __lt__() to keep the words sorted
        # by their corresponding sequences' probability
        return self.log_prob<other.log_prob
```

#### 1.2.4 Implementation Detail (II): Step 1. Initialization

`Beam search` is implemented in the function `greedyDecoder`. First, we provide an additional parameter, `BeamSearchWidth`, for the function `greedyDecoder`:

```
def greedyDecoder(decoder, encoder_out, encoder_hidden, maxLen,
                  eos_index, BeamSearchWidth = 25):
```

We use a `list`, `candidate_list`, to store the top `num_candidate` candidate sequences. Each candidate sequence is represented as an `Entity` object.

```
candidate_list=[]
num_candidate=BeamSearchWidth

```

Then, we `loop` through the time steps `t`. Recall that our implementation consists of 3 parts:

1) For the time step $t=1$, find the $k$ most probable words $x_{1,1},...,x_{1,k}$;

2) Given that we have the $k$ most probable sequences at time step $t$, generate the $k^{2}$ candidate words at time step $t+1$. Choose the $k$ most probable words from the $k^{2}$ candidate words.

3) Given the $k$ most probable words at time step $t+1$, compute the $k$ most probable sequences at time step $t+1$.





For `t=0`, we generate the top `num_candidate` most probable words first:

```
for t in range(maxLen):
    if t == 0:
        # generate the output
        output, decoder_hidden, alpha = decoder(output, encoder_out, decoder_hidden)
        # "normalize" the output as probability distribution
        log_prob=torch.log(torch.nn.functional.softmax(output, dim=-1)) 
        # find the most probable words
        candidates=output.data.topk(num_candidate)
        # for each of the top candidate words, create an Entity and add it to candidate_list
        for i in range(num_candidate):
            entry=Entry([],[],[],log_prob[0][0][candidates[1][0][0][i].item()].item())
            # out is the token id of the candidate word
            out=torch.empty((1,1),dtype=torch.int64)
            out[0][0]=candidates[1][0][0][i].item()
            entry.idx_seq.append(out)
            # store the hidden states in a sequence
            entry.hidden_state_seq.append(decoder_hidden)  
            # store the alpha vector in a sequence
            entry.alpha_seq.append(alpha)
            # add the candidate word to the list
            candidate_list.append(entry)
```

#### 1.2.5 Implementation Detail (III): Step 2. Recursion

Then, for `t > 0`, we generate the $k^{2}$ candidate words. We use `Prob_Rank` to represent these $k^{2}$ candidate words. In `Prob_Rank`, we record the candidate word's token id (`Prob_Rank.idx`), the hidden state (`Prob_Rank.hidden_state`), the $\alpha$ vector (`Prob_Rank.alpha`), the joint probability (`Prob_Rank.log_prob`) of the word and its previous sequence, we also use `Prob_Rank.prev` to keep track of the sequence at the previous time step which generates the current candidate word. We push all `Prob_Rank` objects to a `min Heap` which pops the sub-optimal words out and only stores the top $k$ candidates.

Still we need to take special treatment for the sequences which has encountered the `EOS` token in previous time steps. For such sequences, we assume that it has been terminated. If such sequence can stay on the top candidate list, for each time step, we just add a `dummy` Prob_Rank object which doesn't affect the probability estimation of the sequence.

For the "normal" sequences which haven't encountered the `EOS` token, we need to generate the top $k$ probable words for each of them. To reduce some computational cost, we always store only the top $k$ words as `Prob_Rank` objects in the `min Heap`. When we generate a new word with the `topk` method, we compare its probability with the minimum element in the `min Heap`. If the probability of the new word is greater than the minimum element in the `min Heap`, we pop the minimum element in the `min Heap` out and push the new word to the `min Heap`. If the probability of the new word is no greater than the minimum element in the `min Heap`, then we don't do any thing about the `min Heap`. As we check the words with `descending` likelihood with `topk` method, when we find one word in `topk` list that is not as good as elements in `min Heap`, we can ignore all the rest words in `topk` list.

Below is a possible implementation:

```
# expanded_candidate_list: store the top k words at time step t+1. Organized as min Heap.
expanded_candidate_list=[]
# next_candidate_list is for the k candidate sequences at time step t+1
# (recall that k candidate sequences at time step t are stored in candidate_list)
next_candidate_list=[]
# store the top k words at time step t+1 in a min Heap. Be ready to pop out the element with minimum probability
heapq.heapify(expanded_candidate_list)
for i in range(num_candidate):
    # we provide special treatment for previous candidate sequences which has been terminated by EOS
    if(int(candidate_list[i].idx_seq[-1].data)==eos_index):
        # In such cases, we create a dummy node. The joint probability (of this dummy word and the previous 
        # sequence) is the probability of the previous sequence.
        prob_record=Prob_Rank(i,None,None,None,candidate_list[i].log_prob)
        # we only store at most num_candidate candidate words in the min Heap.
        # if the min Heap is not "full", push the word record to the heap
        if(len(expanded_candidate_list)<num_candidate):
            heapq.heappush(expanded_candidate_list,prob_record)
        # if the min Heap is already "full" with num_candidate records, we compare the word record's probability
        # with the minimum element in the min Heap
        else:
            # if the word record is sub-optimal, we do nothing about the min Heap.
            if (prob_record.log_prob<=min(expanded_candidate_list).log_prob):
                continue
            # if the word is better than some records in the min Heap, we pop the minimum element out from the 
            # min Heap, and we push the word into the min Heap
            else:
                heapq.heappop(expanded_candidate_list)
                heapq.heappush(expanded_candidate_list,prob_record)
    # for the "normal" sequences without an EOS
    else:
        output, decoder_hidden, alpha = decoder(candidate_list[i].idx_seq[-1], encoder_out, 
                                                candidate_list[i].hidden_state_seq[-1])
        log_prob=torch.log(torch.nn.functional.softmax(output, dim=-1))
        # find the top k candidate words from the output distribution
        candidates=output.data.topk(num_candidate)
        # for each candidate word, create a Prob_Rank record, check if we need to push that to the min Heap
        # of the top k words
        for j in range(num_candidate):
            out=torch.empty((1,1),dtype=torch.int64)
            out[0][0]=candidates[1][0][0][j].item()
            # keep record of the word token id, hidden states, alpha vector, and update the joint probability
            prob_record=Prob_Rank(i,out,decoder_hidden,alpha,candidate_list[i].log_prob+
                                log_prob[0][0][candidates[1][0][0][j].item()].item())
            # if the min Heap is not "full", add the record to the min Heap
            if(len(expanded_candidate_list)<num_candidate):
                heapq.heappush(expanded_candidate_list,prob_record)
            # if the min Heap is "full", compare the record with the minimum element in the min Heap
            else:
                # if the record is sub-optimal, then all the rest records in topk are sub-optimal
                # and we need to skip them
                if (prob_record.log_prob<=min(expanded_candidate_list).log_prob):
                    continue
                # if the record is better than some of the records in the min Heap, pop the minimum element
                # out, and push the record to the min Heap
                else:
                    heapq.heappop(expanded_candidate_list)
                    heapq.heappush(expanded_candidate_list,prob_record)
```



#### 1.2.6 Implementation Detail (IV): Step 3. Update

So far, we obtained the $k$ most probable words at time step $t+1$. Then, we need to merge them with the list of top $k$ sequences at time step $t$ to create the list of top $k$ sequences at time step $t+1$.

So, in the min Heap of the $k$ most probable words (`expanded_candidate_list`), we start from the one with the highest probability (since it's a min Heap, the maximum element is `expanded_candidate_list[-1]`). We use `expanded_candidate_list[ ].prev` to find the corresponding previous sequence at time step $t$ in `candidate_list`. We add the word's token id, hidden state, and alpha vector to the sequence records.

```
# we check the most probable num_candidate words at time step t+1
for i in range(num_candidate):
    # we start from the most probable word and find its previous sequence: entry
    entry=candidate_list[expanded_candidate_list[-i-1].prev]
    # we create new_entry, which will store the updated sequence at time step t+1
    # we copies entry's (a time step t sequence) sequences of word token ids, hidden states, alpha vectors
    # to new_entry
    # we update the probability in new_entry
    # note that we need to use .copy() method. Or, the append() method will affect both entry and new_entry,
    # which creates serious errors for translation
    new_entry=Entry(entry.idx_seq.copy(),entry.hidden_state_seq.copy(),
                    entry.alpha_seq.copy(),expanded_candidate_list[-i-1].log_prob)
    # if the previous sequence has been terminated by EOS, then make no update
    # or, we append the word's information to the sequence
    if (int(new_entry.idx_seq[-1].data)!=eos_index):
        new_entry.idx_seq.append(expanded_candidate_list[-i-1].idx)
        new_entry.hidden_state_seq.append(expanded_candidate_list[-i-1].hidden_state)
        new_entry.alpha_seq.append(expanded_candidate_list[-i-1].alpha)
    # we use next_candidate_list to store the candidate sequences at time step t+1
    next_candidate_list.append(new_entry)
# now, candidate_list stores the sequences at time step t+1, and next_candidate_list will be emptied
# at the beginning of next iteration to store the sequences at time step t+2
candidate_list=next_candidate_list
```

#### 1.2.7 Implementation Detail (V): Step 4. Final Output

After all iterations, we use the most probable sequence, `candidate_list[0]`, to provide the decoding output for the final translation:

```
for t in range(maxLen):
    alphas[t]=candidate_list[0].alpha_seq[t].data
    output=candidate_list[0].idx_seq[t]
    outputs[t]=torch.zeros(outputs[t].shape)
    outputs[t][0][output[0][0].item()]=1
    if int(output.data) == eos_index:
        break           
```

### 1.3 Post Processing: Replication Removal<a name='sec_1_3'></a>
[back to content](#content)

#### 1.3.1 Why Replication Removal?

We noticed that with `baseline method` and `beam search method`, the translation sometimes contains repetitive words, such as "`when when` i was". This would negatively affect the `BLEU` score (and translation quality), because the referece translation sentences contain virtually no $n$-gram terms with repetitive words such as `when when`.

To improve the `BLEU` score, we proposed to implement the `replication removal` mechanism as a post processing technique in the `translate` function.

#### 1.3.2 Problem Formulation

For `replication removal`, basically what we are doing is:

For the sentence with the sequence of words: $w_{1}, w_{2}, ..., w_{n}$, keep $w_{i}$ ($2\leq i\leq n$) in the post-processed sentence *if and only if* $w_{i}\neq w_{i-1}$.

#### 1.3.3 Implementation

In `translate` function, after we obtain `output` from the decoder, we perform additional post processing to remove repetitive words. We create a list, `post_processed_output`. A word `w` in the `output` will be added to `post_processed_output`, only if `w` is different from its previous word `w_prev`. Hence, we eliminate all repetitive words in `output` and store the processed sentence in `post_processed_output`.

```
post_processed_output=[]
w_prev=None
for w in output.split(" "):
    if w_prev==None or w_prev!=w:
        post_processed_output.append(w)
    w_prev=w
post_processed_output=" ".join(post_processed_output)
results.append(post_processed_output)
return results
```

### 1.4 Post Processing: \<UNK\> Replacement <a name='sec_1_4'></a>
[back to content](#content)

#### 1.4.1 Why \<UNK\> Replacement?

We realized that the \<UNK\> token negatively affects the `BLEU` score, because the reference translation sentences contain no $n$-gram terms with such token.

Many \<UNK\> replacement strategies can improve `BLEU` score. As \<UNK\> has 0 occurrence in reference translation sentences, replacing \<UNK\> with any words that has positive occurrence in reference translation sentences might improve `BLEU` score. So, a simple idea is to remove \<UNK\>, or replace \<UNK\> with some frequent words in English. Such methods can indeed improve `BLEU` score, however they also lead to worsened translation quality. When \<UNK\> is removed or replaced by some random words, the readability of the text decreases.

To improve `BLEU` score without sacrificing the translation quality, we proposed to replace the \<UNK\> token with its corresponding word or words in the original (German) text. This approach has several advantages:

1) it doesn't incur additional information loss. The information in the original text is preserved;

2) Many German words and English words share the same `etymological sources` (`origin of the words`), so an English reader can correctly infer some of the German words' meaning (for example, the German word `psychotherapie-patientin` is "psychological therapy patient");

3) Some of the \<UNK\> tokens might correspond to `people's names`, or `places`. In such cases, leave the word as it is might be the correct translation.

#### 1.4.2 Problem Formulation

In the attention mechanism, for each target word, we have computed the alpha vector, which is the target word's relevance to every source words. Now, for the whole sentence with target words $w_{1},...,w_{n}$, we obtained $n$ alpha vectors, $\alpha_{1},...,\alpha_{n}$. We can organize all $\alpha_{1},...,\alpha_{n}$'s parameters in a matrix $A=(a_{ij})$ so that $a_{ij}$ describes the relevance between the source word $i$ and the target word $j$.

Then, for each source word $i$, we compute $p_{i}$, which describes which target word the $i$-th source word is the most relevant to:

$p_{i}=\underset{j}{argmax}\alpha_{ij}$

When we encounter an \<UNK\> at position `idx` in the target text, we check all $p_{i}$s to see if there is any $p_{i}=idx$. If so, then the $i$-th source word is the most relevant to the `idx`-th target word and we replace the \<UNK\> token in the `idx`-th position by the $i$-th word in the original text.

#### 1.4.3 Implementation

We implemented \<UNK\> replacement in the `translate` function before `replication removal`.

First, we added an `additional` parameter, `srcFile`, for the function `translate`, so that the `translate` function can get access to the original German text without tokenization.

Then, we obtain the `attention` tensor from the decoder. `attention` is a \[$batch(=1)$, $L_{source}$, $L_{target}$\] tensor, which captures how each target word is relevant to which part of the source text (that is, `attention`\[:, :, i\] describes the i+1 th target word's distribution of attention in the source text, wheras `attention`\[:, i, :\] describes how the i+1 th source word is relevant to each of the target word).

We compute `focus` from `attention` by `focus=attention.topk(1)[1]`. Each `focus[i]` describes the location(s) of the target word(s) that the $i$-th source word is most relevant to. When we encounter \<UNK\> tokens in the target text, we check `focus` the find out which source word(s) is(are) most relevant to this \<UNK\> token. Then, we simply replace the \<UNK\> token with the word(s) in the original text.

```
def translate(model, test_iter, srcFile):
    results = []
    # srcFile is the additional parameter that we added, so that the translate function can get access to
    # source text without tokenization
    src = open(srcFile).read().lower().strip().split("\n")
    for i, batch in tqdm(enumerate(test_iter)):
        # attention describes relevance between source word i and target word j
        output, attention = model(batch.src)
        output = output.topk(1)[1]

        # focus describes for each source word, which target word(s) is(are) the most relevant
        focus=attention.topk(1)[1]

        output = model.tgt2txt(output[:, 0].data).strip().split('<EOS>')[0]

        for idx, w in enumerate(output.split(" ")):
            # if the idx-th target word is <unk>
            if w=='<unk>':
                # find out which source words(s) are the most relevant to the idx-th target word
                source_word_location=(focus==idx).nonzero(as_tuple=True)
                w=''
                # replace <unk> with the relevant target word(s) in src (the original text)
                for src_idx in range(len(source_word_location[1])):
                    w+=src[i].split(" ")[source_word_location[1][src_idx]]
```

### 1.5 "Ensemble" of Models <a name='sec_1_5'></a>
[back to content](#content)

#### 1.5.1 Ensemble Decoding

According to https://www2.cs.sfu.ca/~anoop/papers/pdf/cmu-lti-dec7-2012.pdf, `ensemble decoding` means that the final probability estimation is made by a `linear interpolation` of the estimations made by each of the models.

$p(\bar{e}|\bar{f})\propto exp(\overset{M}{\underset{m}{\Sigma}}\lambda_{m}log p_{m}(\bar{e}|\bar{f}))$

Here, we have $M$ models, and $\lambda_{m} (1\leq m\leq M)$ are the weights for the models. Each of the $M$ models made a probability estimation $p_{m}(\bar{e}|\bar{f})$, and we calculate the weighted sum:

$\overset{M}{\underset{m}{\Sigma}}\lambda_{m}log p_{m}(\bar{e}|\bar{f})$

Then, we need to use `softmax` to "normalize" the $\overset{M}{\underset{m}{\Sigma}}\lambda_{m}log p_{m}(\bar{e}|\bar{f})$ values into probabilities:

$p(\bar{e}|\bar{f})=\frac{exp(\overset{M}{\underset{m}{\Sigma}}\lambda_{m}log p_{m}(\bar{e}|\bar{f}))}{\underset{\bar{e}\in V}{\Sigma}exp(\overset{M}{\underset{m}{\Sigma}}\lambda_{m}log p_{m}(\bar{e}|\bar{f}))} $

If we use this probability estimation function $p(\bar{e}|\bar{f})$ for our decoding process, then we are performing `ensemble decoding`.

#### 1.5.2 Our "Ensemble" of Models

In our work, we **didn't** implement the above `ensemble decoding` process. In our implementation, we **didn't** use `ensembling` techniques during `decoding`, instead, we used the "ensemble" of models during the `translation` stage.

##### Method

Assume that we have $M$ models, and each of the $M$ models have generated a decoded sequence $O_{i} (1\leq i\leq M)$ and a sequence of attention (alpha) vectors $A_{i} (1\leq i\leq M)$. What we are doing is to calculate the linear combination of $O_{i}$s and $A_{i}$s:

$O=\overset{M}{\underset{i=1}{\Sigma}}\lambda_{i}O_{i}, A=\overset{M}{\underset{i=1}{\Sigma}}\lambda_{i}A_{i}$

Here, $\lambda_{i} (1\leq i\leq M)$ is the weights for the model $i$. Instead of using one $O_{i}$ and $A_{i}$ from a single model, we use the linear interpolation $O$ and $A$, where $O$ predicts the target words at each location, and $A$ predicts which part of the source text is relevant to which target word. Since we don't use linear interpolation in the decoding process, we are **not** implementing `ensemble decoding`. However, use did use a linear interpolation of model predictions to guide the `translation` process, which is why we still think that our approach adopts "ensemble" of models.

##### Implementation

Our implementation is as follows.

First, in the `main` function, we need to load multiple models and define the `weights`:

```
model1=Seq2Seq(build=False)
model1.load(os.path.join('data','seq2seq_E049.pt'))
model1.to(hp.device)
model1.eval()
model2=Seq2Seq(build=False)
model2.load(os.path.join('data','seq2seq_E048.pt'))
model2.to(hp.device)
model2.eval()
model3=Seq2Seq(build=False)
model3.load(os.path.join('data','seq2seq_E047.pt'))
model3.to(hp.device)
model3.eval()
model4=Seq2Seq(build=False)
model4.load(os.path.join('data','seq2seq_E046.pt'))
model4.to(hp.device)
model4.eval()
model5=Seq2Seq(build=False)
model5.load(os.path.join('data','seq2seq_E045.pt'))
model5.to(hp.device)
model5.eval()
test_iter=loadTestData(opts.input,model1.fields['src'],device=hp.device,linesToLoad=opts.num)
models=[model1,model2,model3,model4,model5]
weights=[0.5,0.25,0.125,0.08,0.045]
results=translate(models,weights,test_iter,opts.input)
print("\n".join(results))
```

Then, we need to modify the parameters of the `translate` function so that it takes an array of models (the variable `models`) and their associated weights (the variable `weights`) as inputs:

```
def translate(models, weights, test_iter, srcFile):
```

Then, we modify the calculation of `output` $O$ and `attention` $A$ in the `translate` function. We wrote a loop to get the `output` and `attention` from each model, and we calculate a linear interpolation with `weights` to get the final `output` and `attention`.

```
output=None
attention=None
# "ensemble" of models
for model, weight in zip(models, weights):
    # get each model's output and attention
    output_m, attention_m = model(batch.src)
    # multiply each model's output and attention with their weights, then add up
    if output==None and attention==None:
        output=weight*output_m
        attention=weight*attention_m
    else:
        output+=weight*output_m
        attention+=weight*attention_m
```

## 2. Experiments <a name='sec_2'></a>
[back to content](#content)

### 2.1 Baseline Method <a name='sec_2_1'></a>
[back to content](#content)

#### 2.1.1 Qualitative Results on Dev Dataset

In [3]:
os.chdir(directory)
from neuralmt_baseline import *

In [4]:
os.chdir('../')
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results)[:4999])

1305it [04:01,  5.39it/s]

when when i was in my 20s , i had my first <unk> . . 
i was a and i was particularly in berkeley . berkeley . 
she was a a woman woman named alex . 
when the came came up the first first , she came up and she a a a , and she fell into the couch in my office , <unk> her her and and told me , about about about . 
and when i heard this , i was was . 
my my , , had a a patient patient for a first patient . 
and i got a woman in the people who wanted to talk about about . 
i 'll leave that , i thought , . 
but i did n't have it . 
with the the stories , that alex opened into the , was easy easy to me just to with with head , while we were the problems . 
" 30 is the new 20 " , " alex , and as far as i did , i had right . 
work came later later , later came later later , later came later later later later came later later . 
people people in the the , and alex , and i did n't have time . 
but soon my my my was , my head , to ask . 
i i was going to . 
i said , " yeah , you know , you 're wit

#### 2.1.2 Quantitative Results on Dev Dataset

In [6]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))
print(bleu(ref_t, results).score)

BLEU = 17.11 54.0/24.1/12.0/6.5 (BP = 0.957 ratio = 0.958 hyp_len = 23858 ref_len = 24902)
17.11393268339238


### 2.2 Beam Search <a name='sec_2_2'></a>
[back to content](#content)

In this experiment, we set the `beam width` parameter as 5.

#### 2.2.1 Qualitative Results on Dev Dataset

In [7]:
os.chdir(directory)
from neuralmt_beam_search import *

In [8]:
os.chdir('../')
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results)[:4999])

1305it [20:35,  1.06it/s]

when when i was in my 20s , i had my first <unk> . . 
i was a , and i studied in berkeley . berkeley . 
she was a woman woman named alex . 
as soon as the first arrived came , , she came up and she a a a , and she fell into the couch in my office , <unk> her , and she told me to talk about about <unk> . 
and when i heard this , i was was . . 
i mean , my <unk> had had a a patient as a first patient . 
and i got a woman in the people who wanted to talk about about . . 
i 'll leave it , i thought i was . 
but i did n't have it . 
with the the stories , the alex came into the the , it was easy easy to me , just to head with head , as we faced the problems . . 
" 30 is the new 20 " said , " alex , and as far as i did , i was right . 
work came later later , later came later , later came later later , later came later later . 
people people in the , and alex , and i did n't have time time . 
but soon , my my , my god was asking in question . 
i i did it . 
i said , " yeah , yeah , you 'll b

#### 2.2.2 Quantitative Results on Dev Dataset

In [9]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))
print(bleu(ref_t, results).score)

BLEU = 19.04 54.6/25.9/13.5/7.6 (BP = 0.978 ratio = 0.978 hyp_len = 24354 ref_len = 24902)
19.03769045192745


### 2.3 Replication Removal <a name='sec_2_3'></a>
[back to content](#content)

In this experiment, we tested the combination of `beam search` and `replication removal`. The `beam width` parameter is set as 5.

#### 2.3.1 Qualitative Results on Dev Dataset

In [12]:
os.chdir(directory)
from neuralmt_beam_search_with_replication_removal import *

In [13]:
os.chdir('../')
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results)[:4999])

1305it [20:14,  1.07it/s]

when i was in my 20s , i had my first <unk> . 
i was a , and i studied in berkeley . berkeley . 
she was a woman named alex . 
as soon as the first arrived came , she came up and she a , and she fell into the couch in my office , <unk> her , and she told me to talk about <unk> . 
and when i heard this , i was . 
i mean , my <unk> had a patient as a first patient . 
and i got a woman in the people who wanted to talk about . 
i 'll leave it , i thought i was . 
but i did n't have it . 
with the stories , the alex came into the , it was easy to me , just to head with head , as we faced the problems . 
" 30 is the new 20 " said , " alex , and as far as i did , i was right . 
work came later , later came later , later came later , later came later . 
people in the , and alex , and i did n't have time . 
but soon , my , my god was asking in question . 
i did it . 
i said , " yeah , yeah , you 'll be close with your , you with a , but she 's not going to marry it . " 
and since said , " my , 

#### 2.3.2 Quantitative Results on Dev Dataset

In [14]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))
print(bleu(ref_t, results).score)

BLEU = 19.39 60.6/29.2/15.8/9.1 (BP = 0.863 ratio = 0.872 hyp_len = 21715 ref_len = 24902)
19.386790425063992


### 2.4 \<UNK\> Replacement <a name='sec_2_4'></a>
[back to content](#content)

In this experiment, we tested the combination of `beam search`, `replication removal`, and `<UNK> replacement`. The `beam width` parameter is set as 5.

#### 2.4.1 Qualitative Results on Dev Dataset

In [3]:
os.chdir(directory)
from neuralmt_beam_search_with_replication_removal_with_unk_replacement import *

In [4]:
os.chdir('../')
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter, os.path.join('data', 'input', 'dev.txt')) # Warning: will take >5mins depending on your machine
print("\n".join(results)[:4999])

1305it [20:09,  1.08it/s]

when i was in my 20s , i had my first psychotherapie-patientin . 
i was a , and i studied in berkeley . berkeley . 
she was a woman named alex . 
as soon as the first arrived came , she came up and she a , and she fell into the couch in my office , schleuderte her , and she told me to talk about  . 
and when i heard this , i was . 
i mean , my kommilitonin had a patient as a first patient . 
and i got a woman in the people who wanted to talk about . 
i 'll leave it , i thought i was . 
but i did n't have it . 
with the stories , the alex came into the , it was easy to me , just to head with head , as we faced the problems . 
" 30 is the new 20 " said , " alex , and as far as i did , i was right . 
work came later , later came later , later came later , later came later . 
people in the , and alex , and i did n't have time . 
but soon , my , my god was asking in question . 
i did it . 
i said , " yeah , yeah , you 'll be close with your , you with a , but she 's not going to marry it . 

#### 2.4.2 Quantitative Results on Dev Dataset

In [5]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))
print(bleu(ref_t, results).score)

BLEU = 19.71 61.1/29.7/16.2/9.3 (BP = 0.862 ratio = 0.870 hyp_len = 21675 ref_len = 24902)
19.712473363165547


### 2.5 "Ensemble" of Models <a name='sec_2_5'></a>
[back to content](#content)

In this experiment, we tested the combination of `beam search`, `replication removal`, `<UNK> replacement`, and `ensemble of models`. The `beam width` parameter is set as 5.

#### 2.5.1 Qualitative Results on Dev Dataset

In [2]:
os.chdir(directory)
from neuralmt_ensemble import *

In [3]:
os.chdir('../')
model1=Seq2Seq(build=False)
model1.load(os.path.join('data','seq2seq_E049.pt'))
model1.to(hp.device)
model1.eval()
model2=Seq2Seq(build=False)
model2.load(os.path.join('data','seq2seq_E048.pt'))
model2.to(hp.device)
model2.eval()
model3=Seq2Seq(build=False)
model3.load(os.path.join('data','seq2seq_E047.pt'))
model3.to(hp.device)
model3.eval()
model4=Seq2Seq(build=False)
model4.load(os.path.join('data','seq2seq_E046.pt'))
model4.to(hp.device)
model4.eval()
model5=Seq2Seq(build=False)
model5.load(os.path.join('data','seq2seq_E045.pt'))
model5.to(hp.device)
model5.eval()
test_iter=loadTestData(os.path.join('data', 'input', 'dev.txt'),model1.fields['src'],device=hp.device,linesToLoad=sys.maxsize)
models=[model1,model2,model3,model4,model5]
weights=[0.5,0.25,0.125,0.08,0.045]
results=translate(models,weights,test_iter,os.path.join('data', 'input', 'dev.txt'))
print("\n".join(results)[:4999])

1305it [2:41:14,  7.41s/it] 

when i was in my 20s , i had my first psychotherapie-patientin . 
i was a , and i studied in berkeley . berkeley . 
she was a woman named alex . 
as soon as the first arrived came , she came up and she a , and she fell into the office in my office , schleuderte her , and she told me to talk about zu . 
and when i heard this , i was . 
i mean , my kommilitonin had a patient as a first patient . 
and i got a woman in the people who wanted to talk about . 
i 'll leave it , i thought i was . 
but i did n't have it . 
with the stories , the alex came into the , it was easy to me , to with head , we faced the problems . 
" 30 is the new 20 " , " , and as i , i was right . 
work came later , later came later , later came later , later came later . 
people in the , and alex , and i did n't have time . 
but soon , my , my god was asking in question . 
i did it . 
i said , " yeah , yeah , you 'll be close with your , you with a , but she 's not going to marry it . " 
and since said , " , " not y

#### 2.5.2 Quantitative Results on Dev Dataset

In [4]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))
print(bleu(ref_t, results).score)

BLEU = 19.23 61.3/29.5/15.9/9.2 (BP = 0.848 ratio = 0.859 hyp_len = 21384 ref_len = 24902)
19.227925009919375


### 2.6 Final Results <a name='sec_2_6'></a>
[back to content](#content)

In this experiment, we reported our `best-so-far` result. We used a combination of `beam search`, `replication removal`, and `<UNK> replacement`. The `beam width` parameter is set as 25.

#### 2.6.1 Qualitative Results on Dev Dataset

In [5]:
os.chdir(directory)
from neuralmt import *

In [6]:
os.chdir('../')
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter, os.path.join('data', 'input', 'dev.txt')) # Warning: will take >5mins depending on your machine
print("\n".join(results)[:4999])

1305it [1:49:06,  5.02s/it]

when i was in my 20s , i had my first psychotherapie-patientin . 
i was a , and i studied in berkeley . berkeley . 
she was a woman named alex . 
as soon as the first arrived came , she came up , and she was a , and she fell into the street in my office , schleuderte her , and she told me to talk about  . 
and when i heard this , i was  . 
i mean , my kommilitonin had a patient as a first patient . 
and i got a woman in the people who wanted to talk about . 
i 'll leave it , i thought . 
but i did n't have it . 
with the stories , the alex came into the , it was easy to me , just to deal with head , as we faced the problems . 
" 30 is the new 20 " , " alex , and as far as i did , they had right . 
work came later , later came later , later came later , later came later . 
people in the , like alex , and i did n't have time . 
but soon , my , my god was asking in question . 
i was holding . 
i said , " yeah , yeah , you 'll be close with your , you with a , but she 's not going to marry

#### 2.6.2 Quantitative Results on Dev Dataset

In [7]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))
print(bleu(ref_t, results).score)

BLEU = 20.31 61.0/30.0/16.6/9.7 (BP = 0.871 ratio = 0.878 hyp_len = 21875 ref_len = 24902)
20.30719054449544


## 3. Analysis <a name='sec_3'></a>
[back to content](#content)

### 3.1 Attention is Crucial for Neural Machine Translation

From both qualitative and quantitative results we can confirm that attention mechanism greatly improves the
translation quality measured either by BLEU scores or human evaluation. Without attention mechanism, the default
method can still make correct translations for some of the words in the sentence (for example, the default method
correctly translated “I was” in the sentence “When I was in my 20s …”, and it correctly translated “she was a woman”
in the sentence “She was a 26-year-old woman named Alex”). However, without attention, the default method seems
to only be able to recognize some of the most frequent words or phrases in English, and it makes a lot of repetitive
translations. The rich information in the sentence, is not attended to by the model and lost. From such results we can
conclude that, without attention, the encoder’s last hidden state is a bottleneck of information, which can’t provide a
good summary of the source sentence. With attention mechanism, the model can attend to the most relevant part in
the source text for the translation of each target words, which greatly improves the quality of neural machine
translation.

### 3.2 Larger Beam Search Width is Not Always Good

From the quantitative results we can see that the BLEU score peaks when the beam width is around $k$ = 25. After the
beam search width reaches $k$ = 25, further increasing the beam search width helps little for improving BLEU scores.
In fact, it is a little bit counter-intuitive that increasing $k$ from 25 to 30 actually leads to lower BLEU scores. Our
preliminary explanation for this is that, while increase the beam search width ensures that the algorithm can search
over a larger space to find the “optimal” solution, the “optimality” defined by the training dataset might be different
from the “optimality” required in the testing set. As we increase the beam search width, the algorithm makes a more
thorough search, and the solution might converge to the optimality according to the training set, however, this doesn’t
guarantee that the solution is optimal for the testing dataset

### 3.3 Translation Quality is Different From BLEU Scores

If the translated text contains some tokens or patterns of tokens which would never appear in the reference translation
text, then such tokens or patterns of tokens would negatively affect the BLEU score. Removing such tokens, or
replacing such tokens with some frequent words in English, can always be helpful for improving the BLEU score.
However, such technique might actually leads to worsened translation quality, because the text can become confusing
after removal of tokens or replacement of the tokens with some random words. What we learned from the experiments
is that we still want to replace such tokens with some relevant information, so that we can preserve the information in
the text, and improve the BLEU score without sacrificing the translation quality.

### 3.4 Why Ensemble Fails?

In our experiments, we tested the “ensemble” of models, however, the result of ensemble is not as good as the same
method without “ensemble”. We have 2 preliminary explanations for this: 1) our implementation of “ensemble” is not
good, and its result is still far from the results that we can get from ensemble decoding. 2) we guess that we need to
use a number of fundamentally different models, so that the ensemble of models can produce a good results. In our
experiments, all models are using the same network architecture, and we guess that their only possible differences are
the training epochs and parameters. Since all models use the same network architecture, they share the same
advantages and drawbacks. Since their only difference might be the training epochs, maybe in the 5 models, some are
just better than others because they are trained with more epochs. We guess that in ensemble learning, we actually
want the models to have different advantages, so that they can cover each other’s drawbacks. In our experiments, all
5 models are “homogenous” with the same network architecture, same advantages, and same drawbacks, which might
be a reason why ensemble of such models won’t produce good results.